Импортирую кучу необходимых библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

Загружаю данные датасета и вывожу 5 первых строк

In [9]:
data = pd.read_csv('titanic.csv')
data.head()

,PassengerId,Survived,Pclass,Lname,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Braund,Mr. Owen Harris,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,Heikkinen,Miss. Laina,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,Allen,Mr. William Henry,male,35.0,0,0,373450,8.0500,NaN,S


Просматриваю общую информацию о датасете

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156 entries, 0 to 155
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  156 non-null    int64  
 1   Survived     156 non-null    int64  
 2   Pclass       156 non-null    int64  
 3   Lname        156 non-null    object 
 4   Name         156 non-null    object 
 5   Sex          156 non-null    object 
 6   Age          126 non-null    float64
 7   SibSp        156 non-null    int64  
 8   Parch        156 non-null    int64  
 9   Ticket       156 non-null    object 
 10  Fare         156 non-null    float64
 11  Cabin        31 non-null     object 
 12  Embarked     155 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 16.0+ KB


Проверяю на пустые значения

In [11]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Lname            0
Name             0
Sex              0
Age             30
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          125
Embarked         1
dtype: int64

Решаю удалить колонку 'Cabin', а то, много как-то пустых значений

In [12]:
data.drop('Cabin', axis=1, inplace=True)

В колонке 'Age' решил заполнить пустые значения медианой

In [13]:
data['Age'].fillna(data['Age'].median(), inplace=True)

Ну и с колонкой 'Embarked', решил заполнить самыми часто встречающимися значениями

In [20]:
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

Кодирую данные чтобы в дальнейшем можно их было подразделить на обучающие и тестовые наборы... Вот

In [39]:
le = LabelEncoder()
data['Sex'] = le.fit_transform(data['Sex'])
data['Embarked'] = le.fit_transform(data['Embarked'])
data['Lname'] = le.fit_transform(data['Lname'])
data['Name'] = le.fit_transform(data['Name'])
data['Ticket'] = le.fit_transform(data['Ticket'])

Ну собственна разбиваю данные на тестовые и обучающие

In [40]:
X = data.drop('Survived', axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Создаю модель градиентного бустинга

In [41]:
gb = GradientBoostingClassifier(n_estimators=100, max_depth=3, random_state=42)

Обучаю модель на обучающем наборе(X_train, y_train)

In [42]:
gb.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

Играем в экстрасенсов и предсказываем значения для тестового набора

In [43]:
y_pred_gb = gb.predict(X_test)

Вычисляю и вывожу точность "экстрасенсорных способностей"

In [44]:
accuracy_gb = accuracy_score(y_test, y_pred_gb)
print("Точность градиентного бустинга: ", accuracy_gb)

Точность градиентного бустинга:  0.78125


Вычисляю матрицу ошибок

In [45]:
cm_gb = confusion_matrix(y_test, y_pred_gb)
print("Матрица ошибок градиентного бустинга: \n", cm_gb)

Матрица ошибок градиентного бустинга: 
 [[18  3]
 [ 4  7]]


Создаю модель для логистической регрессии

In [46]:
lr = LogisticRegression(random_state=42)

Обучаю модель на обучающем наборе

In [47]:
lr.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

Снова играем в экстрасенсов и предсказываем значения для тестового набора, только уже для логистической регрессии

In [48]:
y_pred_lr = lr.predict(X_test)

Вывожу точность предсказаний

In [49]:
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print("Точность логистической регрессии: ", accuracy_lr)

Точность логистической регрессии:  0.71875


Вывожу матрицу ошибок

In [50]:
cm_lr = confusion_matrix(y_test, y_pred_lr)
print("Матрица ошибок логистической регрессии: \n", cm_lr)

Матрица ошибок логистической регрессии: 
 [[17  4]
 [ 5  6]]


Создаю модель решающего дерева

In [51]:
dt = DecisionTreeClassifier(random_state=42)

Обучаю модель на обучающем наборе

In [52]:
dt.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

Снова играем в экстрасенсов

In [53]:
y_pred_dt = dt.predict(X_test)

Вывожу точность предсказаний

In [54]:
accuracy_dt = accuracy_score(y_test, y_pred_dt)
print("Точность решающего дерева: ", accuracy_dt)

Точность решающего дерева:  0.75


Вывожу матрицу ошибок

In [55]:
cm_dt = confusion_matrix(y_test, y_pred_dt)
print("Матрица ошибок решающего дерева: \n", cm_dt)

Матрица ошибок решающего дерева: 
 [[19  2]
 [ 6  5]]


Ну и сравниваю все это вместе для вида

In [56]:
print("Точность градиентного бустинга: ", accuracy_gb)
print("Матрица ошибок градиентного бустинга: \n", cm_gb)

print("Точность логистической регрессии: ", accuracy_lr)
print("Матрица ошибок логистической регрессии: \n", cm_lr)

print("Точность решающего дерева: ", accuracy_dt)
print("Матрица ошибок решающего дерева: \n", cm_dt)

Точность градиентного бустинга:  0.78125
Матрица ошибок градиентного бустинга: 
 [[18  3]
 [ 4  7]]
Точность логистической регрессии:  0.71875
Матрица ошибок логистической регрессии: 
 [[17  4]
 [ 5  6]]
Точность решающего дерева:  0.75
Матрица ошибок решающего дерева: 
 [[19  2]
 [ 6  5]]


Вывод:
По итогу можно сказать, что самым точным для данного датасета оказался градиентный бустинг 78%, против логистической регресси 71 % и решающего дерева 75%.